In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../datasets/MNIST_data/", one_hot=True)

In [1]:
import tensorflow as tf
v1 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
v2 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\model2.ckpt")

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
# 载入MNIST数据集，如果指定地址/path/to/MNIST_data 下没有已经下载好的数据，那么TensorFlow会自动下载
mnist = input_data.read_data_sets("C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data", one_hot=True)
# 打印Training data
print("Training data size：{}".format(mnist.train.num_examples))
print("Validating data size:{}".format(mnist.validation.num_examples))
print("Testing data size:{}".format(mnist.test.num_examples))
print("Example training data:{}".format(mnist.train.images[0]))
print("Example training data label:{}".format(mnist.train.labels[0]))

In [ ]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)
# 从train的集合中选取batch_size个训练数据
print("X shape:{}".format(xs.shape))
print("Y shape:{}".format(ys.shape))

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# MNIST数据集相关的常数
INPUT_NODE = 784 #输入层的节点数。对于MNIST数据集，这个就等于图片的像素。
OUTPUT_NODE=10 #输出层的节点数。这个等于类别的数目。因为在MNIST数据集中需要区分的是0~9这10个数字，所以这里输出层的节点数为10
# 配置神经网络的参数
LAYER1_NODE = 500 # 隐藏层节点数。这里使用只有一个隐藏层的网络结构作为样例。这个隐藏层有500个节点。
BATCH_SIZE = 100 # 一个训练batch中的训练数据个数。数字越小时，训练过程越接近随机梯度下降；数字越大时，训练越接近梯度下降

LEARNING_RATE_BASE = 0.8 # 基础学习率
LEARNING_RATE_DECAY = 0.99 # 学习率的衰减率
REGULARIZATION_RATE = 0.0001 # 描述模型复杂度的正则化项在损失函数中的系数
TRAINING_STEPS = 30000 # 训练轮数
MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减率

# 一个辅助函数，给定神经网络的输入和所有参数，计算神经网络的前向传播结果。在这里定义了一个使用ReLu激活函数的三层
# 全连接神经网络。通过加入隐藏层实现了多层网络结构，通过ReLU激活函数实现了去线性化。在这个函数中也支持传入用于计算参数平均值的类，
# 这样方便在测试时使用滑动平均模型。
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值。
    if avg_class == None:
        # 计算隐藏层的前向传播结果，这里使用了ReLU激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        # 计算输出层的前向传播结果。因为在计算损失函数时会一并计算softmax函数，
        # 所以这里不需要加入激活函数。而且不加入Softmax不会影响预测结果。因为预测时
        # 使用的是不同类别对应节点输出值的相对大小，有没有Softmax层对最后分类结果的
        # 计算没有影响。于是在计算整个神经网络的前向传播时可以不加入最后的Softmax层。
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先使用avg_class.average函数来计算得出变量的滑动平均值
        # 然后再计算相应的神经网络前向传播结果。
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name = 'x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name = 'y-input')
    # 生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    # 生成输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算在当前参数下神经网络前向传播的结果。这里给出的用于计算滑动平均的类为None，
    # 所有函数不会使用参数的滑动平均值
    y = inference(x,None,weights1,biases1,weights2,biases2)
    # 定义存储训练轮数的变量。这个变量不需要计算滑动平均值，所以这里指定这个变量为
    # 不可训练的变量（trainable=False）。在使用TensorFlow训练神经网络时，
    # 一般会将代表训练轮数的变量指定会不可训练的参数
    gloabal_step = tf.Variable(0,trainable=False)
    
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类。
    # 在上一章介绍过给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    # 在所有代表神经网络参数的变量上使用滑动平均。其他辅助变量（比如global_step）就不需要了。
    # tf.trainable_variables返回的就是图上集合
    # GraphKeys.TRAINABLE_VARIABLES中的元素。这个集合的元素就是所有没有指定trainable=False的参数。
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用了滑动平均之后的前向传播结果。上一章中介绍过滑动平均不会改变变量本身的取值，而是会维护一个影子
    # 变量来记录滑动平均值。所以当需要使用这个滑动平均值时，需要明确调用average函数。
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数。
    #这里使用了TensorFlow中提供的sparse_softmax_cross_entropy_with_logits函数来计算交叉熵。当分类问题只有
    #一个答案时，可以使用这个函数来加速交叉熵的计算。MNIST问题的图片中只包含了0~9中的数字，所以可以使用
    #这个函数来计算交叉熵损失。这个函数的第一个参数是神经网络不包括Softmax层的前向传播结果，
    #第二个是训练数据的正确答案。因为标准答案是一个长度为10的一维数组，而该函数需要提供的是一个正确答案的数字，
    #所以需要使用tf.argmax函数来得到正确答案对应的类别编号。
    cross_entropy = tf.nn.sparse_softmax_cross_entorpy_with_logits(y, tf.argmax(y_,1))
    # 计算当前batch中所有样例的交叉熵平均值。
    cross_entropy_mean = tf.reduce_mean(cross_entroy)
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失。一般只计算神经网络边上权重的正则化损失，而不使用偏置项。
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失等于交叉熵损失和正则化损失的和。
    loss = cross_entropy_mean + regularization
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples/BATCH_SIZE, LEARNING_RATE_DECAY)
    # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数。注意这里损失函数包含了交叉熵损失和L2正则化损失。
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # 在训练神经网络模型时，每过一遍数据既需要通过反向传播来更新神经网络中的参数，又要更新每一个参数的滑动平均值。
    # 为了一次完成多个操作，TensorFlow提供了tf.control_dependencies和tf.group两种机制。下面两行程序和
    # train_op = tf.group(train_step, variables_averages_op)是等价的。
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name='train')
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确。tf.argmax(average_y,1)计算每一个样例的预测答案。
    # 其中average_y是一个batch_size*10的二维数组，每一行表示一个样例的前向传播结果。tf.argmax的第二个参数“1”表示
    # 选取最大值的操作仅在第一个维度中进行，也就是说，只在每一行选取最大值对应的下标。于是得到的结果是一个长度为batch
    # 的一维数组，这个一维数组中的值就表示了每一个样例对应的数字识别结果。tf.equal判断两个张量的每一维是否相等，如果
    #相等返回true，否则返回False
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    # 这个运算首先将一个布尔型的数值转换为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 准备验证数据。一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和评判训练的效果。
        validate_feed = {x:mnist.validation.images, y_:mnist.validation.lables}
        # 准备测试数据。在真实的应用中，这部分数据在训练时是不可见的，这个数据只是作为模型优劣的最后评价标准。
        test_feed = {x:mnist.test.images, y_:mnist.test.lables}
        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果。
            if i % 1000 == 0:
                #计算滑动平均模型在验证数据上的结果。因为mnist数据集比较小，所以一次可以处理所有的验证数据。
                #为了计算方便，本样例程序没有将验证数据划分为更小的batch。当神经网络模型比较复杂或者验证数据
                #比较大时，太大的batch会导致计算时间过长甚至发生内存溢出的错误。
                validate_acc = sess.run(accuacy, feed_dict = validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g" %(i,validate_acc))
                # 产生这一轮使用的一个batch的训练数据，并运行训练过程
                xs, ys = mnist.train.next_batch(BATCH_SIZE)
                sess.run(train_op, feed_dict={x:xs, y_:ys})
            # 在训练结束之后，在测试数据上监测神经网络模型的最终正确率。
            test_acc = sess.run(accuracy, feed_dict = test_feed)
            print("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc))

# 主程序入口
def main(argv=None):
    # 声明处理MNIST数据集的类，这个类在初始化时会自动下载数据
    mnist = input_data.read_data_sets("C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

In [4]:
import tensorflow as tf
v = tf.get_variable("v_test1", shape=[1], initializer=tf.constant_initializer(1.0))
v2 = tf.Variable(tf.constant(1.0, shape=[1]), name="v_test")
print(v)
print(v2)

<tf.Variable 'v_test1:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'v_test:0' shape=(1,) dtype=float32_ref>


In [5]:
# 在名字为foo的命名空间内创建名字为v的变量
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1], initializer=tf.constant_initializer(1.0))

# 因为命名空间foo中已经存在名字为v的变量，所以下面的代码将会报错：
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])

# 在生成上下文管理器时，将参数reuse设置为True。这样tf.get_variable函数直接获取已经声明的变量。
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v==v1)

# 将参数reuse设置为True时，tf.variable_scope将只能获取已经创建过的变量。
# 因为在命名空间bar中还没有创建变量v，所以下面的代码将会报错：
with tf.variable_scope("bar", reuse=True):
    v = tf.get_variable("v", [1])
    

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2628, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [8]:
with tf.variable_scope("root"):
    # 可以通过tf.get_variable_scope().reuse 函数来获取当前上下文管理器中reuse参数的取值
    print(tf.get_variable_scope().reuse)
    
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        
        with tf.variable_scope("bar"):
            print(tf.get_variable_scope().reuse)
    
    print(tf.get_variable_scope().reuse)

False
True
True
False


In [14]:
# v = tf.get_variable("v", [1])
print(v.name)

# with tf.variable_scope("foo"):
#     v2 = tf.get_variable("v", [1])
#     print(v2.name)
print(v2.name)

with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v", [1])
        print(v3.name)
    v4 = tf.get_variable("v1", [1])
    print(v4.name)
    
# 创建一个名称为空的命名空间，并设置reuse=True。
with tf.variable_scope("", reuse=True):
    v5 = tf.get_variable("foo/bar/v", [1])
    print(v5 == v3)
    v6 = tf.get_variable("foo/v1", [1])
    print(v6==v4)

foo/v:0
v_test:0
foo/bar/v:0
foo/v1:0
True
True


In [34]:
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2
init_op = tf.initialize_all_variables()
# 声明tf.train.Saver类用于保存模型。
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    # 将模型保存
    saver.save(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\model_data\\model.ckpt")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [38]:
# 使用和保存模型代码中一样的方式来声明变量
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\model_data\\checkpoint")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from C:\Users\1003342\Desktop\study\tensorflow\model_data\checkpoint


DataLossError: Unable to open table file C:\Users\1003342\Desktop\study\tensorflow\model_data\checkpoint: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[Node: save_16/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_16/Const_0_0, save_16/RestoreV2_1/tensor_names, save_16/RestoreV2_1/shape_and_slices)]]

Caused by op 'save_16/RestoreV2_1', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3

In [ ]:
saver = tf.train.import_meta_graph("C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data\\model.meta")
with tf.Session() as sess:
    saver.restore(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data\\model.ckpt")
    # 通过张量的名称来获取张量
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

In [39]:
# 这里声明的变量名称和已经保存的模型中变量的名称不同
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="other-v2")

# 如果直接使用tf.train.Saver()来加载模型会报找不到的错误。下面显示了报错信息

# 使用一个字典来重命名变量就可以加载原来的模型了。这个字典指定了原来名称为v1的变量现在加载到变量v1中（名称为other-v1），
# 名称为v2的变量加载到变量v2中（other-v2）
saver = tf.train.Saver({"v1":v1, "v2":v2})

In [40]:
v = tf.Variable(0, dtype=tf.float32, name="v")
# 在没有申明滑动平均模型时只有一个变量v，所以下面的语句只会输出“v:0”
for variables in tf.all_variables():
    print(variables.name)
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.all_variables())
# 在申明滑动平均模型之后，TensorFlow会自动生成一个影子变量
# v/ExponentialMoving Average。于是下面的语句会输出
for variables in tf.all_variables():
    print(variables.name)

saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    # 保存时，TensorFlow会将两个变量保存下来。
    saver.save(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data2\model.ckpt")
    print(sess.run(v, ema.average(v)))

v = tf.Variable(0, dtype=tf.float32, name="v")
# 通过变量重命名将原来变量v的滑动平均值直接赋值给v
saver = tf.train.Saver({"v/ExponentialMovingAverage":v})
with tf.Session() as sess:
    saver.restore(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data2\model.ckpt")
    print(sess.run(v))


Instructions for updating:
Please use tf.global_variables instead.
v:0
v_1:0
v_test1:0
v_test:0
foo/v:0
foo/bar/v:0
foo/v1:0
v1:0
v2:0
v1_1:0
v2_1:0
v1_2:0
v2_2:0
v1_3:0
v2_3:0
v1_4:0
v2_4:0
v1_5:0
v2_5:0
v1_6:0
v2_6:0
v1_7:0
v2_7:0
v1_8:0
v2_8:0
v1_9:0
v2_9:0
v1_10:0
v2_10:0
v1_11:0
v2_11:0
v1_12:0
v2_12:0
v1_13:0
v2_13:0
v1_14:0
v2_14:0
v1_15:0
v2_15:0
v1_16:0
v2_16:0
v1_17:0
v2_17:0
v1_18:0
v2_18:0
v1_19:0
v2_19:0
v1_20:0
v2_20:0
other-v1:0
other-v2:0
v_2:0


In [ ]:
# 为了方便加载时重命名滑动平均变量，tf.train.ExponentialMovingAverage类提供了variables_to_restore函数
# 来生成tf.train.Saver类所需要的变量重命名字典。以下代码给出了variables_to_restore函数的使用样例
import tensorflow as tf
v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
# 通过使用variables_to_restore函数可以直接生成上面代码中提供的字典

# 其中后面的Variable类就代表了变量v
print(ema.variables_to_restore())

saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "C:\\Users\\1003342\\Desktop\\study\\tensorflow\\data2\model.ckpt")
    print(sess.run(v))
